# Test locking on load

In [ ]:
!lamin login testuser2

In [ ]:
import pytest
import lamindb_setup as ln_setup
from lamindb_setup.dev.cloud_sqlite_locker import Locker, InstanceLockedException
from lamindb_setup.dev.upath import UPath

In [ ]:
root = UPath("s3://lndb-setup-ci", cache_regions=True)

In [ ]:
exclusion = root / "exclusion/lndb-load-test-lock"
if exclusion.exists():
    exclusion.rmdir()

In [ ]:
assert ln_setup.settings.user.handle == "testuser2"

In [ ]:
ln_setup.init(storage="s3://lndb-setup-ci", name="lndb-load-test-lock")
ln_setup.close()

In [ ]:
# lock with some random id
locker = Locker("randuseridtt", storage_root=root, name="lndb-load-test-lock")

In [ ]:
locker.lock()

In [ ]:
with pytest.raises(InstanceLockedException):
    ln_setup.load("lndb-load-test-lock")

In [ ]:
locker.unlock()

In [ ]:
ln_setup.load("lndb-load-test-lock")
ln_setup.close()

In [ ]:
# lock through testuser1, who has user id "DzTjkKse"
locker = Locker("DzTjkKse", storage_root=root, name="lndb-load-test-lock")

In [ ]:
locker.lock()

In [ ]:
with pytest.raises(InstanceLockedException) as error:
    ln_setup.load("lndb-load-test-lock")

assert (
    "InstanceLockedException: Cannot load the instance, it is locked by 'testuser1'"
    " (uid: 'DzTjkKse', name: 'Test User1')."
    in error.exconly()
)

In [ ]:
locker.unlock()

In [ ]:
ln_setup.load("lndb-load-test-lock")

In [ ]:
# test ignore_prev_locker=True in unlock_cloud_sqlite_upon_exception
# i.e. test that the locker doesn't unlock if the locker hasn't changed during the function execution
with pytest.raises(RuntimeError):
    ln_setup.load("lndb-load-test-lock")

assert ln_setup.settings.instance._cloud_sqlite_locker._has_lock

In [ ]:
ln_setup.close()

In [ ]:
# purely technical varibale to test failed load after locking
ln_setup._load_instance._TEST_FAILED_LOAD = True

with pytest.raises(RuntimeError):
    ln_setup.load("lndb-load-test-lock")

assert ln_setup.dev.cloud_sqlite_locker._locker._has_lock is None

ln_setup._load_instance._TEST_FAILED_LOAD = False

In [ ]:
ln_setup.load("lndb-load-test-lock", _test=True)

In [ ]:
ln_setup.delete("lndb-load-test-lock", force=True)